In [ ]:
import os
import numpy as np
import pandas as pd
import absl.logging
from nlp_embedding import GloVe, SmallBert
from nlp_classifier2 import NaiveBayes, SVM, XGBoost, CNN
from sklearn import preprocessing, metrics
from ast import literal_eval
absl.logging.set_verbosity(absl.logging.ERROR)

In [ ]:
# Parameters for training and testing

max_words = 400
dataset_name = 'dataset_proc'
cnn_epochs = 10

In [ ]:
def train(data_x, data_y, nlp_embedding, nlp_classifier, label_encoder, batch_size, dataset_name, epochs=1, model_dir='models', start_idx=0, fname_end=''):
    '''
    Train a model.
    Parameters:
        data_x (Series): Lyrics of the training data.
        data_y (Series): Genres of the training data.
        nlp_embedding (NLPEmbedding): Instance of the embedding method class.
        nlp_classifier (NLPClassifier): Instance of the classifier class.
        label_encoder (LabelEncoder): Label encoder after fitting.
        batch_size (int): Number of observations taken for single embedding and partial fitting.
        dataset_name (str): Name of the dataset.
        epochs (int): Number of epochs.
        model_dir (str): Name of directory for saved models.
        start_idx (int): In case of interrupted training, from which observation restart the training.
        fname_end (str): Suffix of the saved model file name.
    '''
    print('Training...')
    fname = os.path.join(model_dir, dataset_name, f'model_{nlp_embedding.name}_{nlp_classifier.name}{fname_end}')
    data_y_enc = label_encoder.transform(data_y)
    classes = np.unique(data_y_enc)
    
    for epoch in range(epochs):
        print(f'Epoch: {str(epoch + 1)}/{str(epochs)}')
        for i in range(start_idx, data_x.shape[0], batch_size):
            
            if i + batch_size > data_x.shape[0]:
                j = data_x.shape[0]
            else:
                j = i + batch_size
            
            print(f'Processing rows: {i} - {j - 1}')

            embeddings = nlp_embedding.embed_lyrics(data_x[i:j])
            nlp_classifier.partial_fit(embeddings, data_y_enc[i:j], classes=classes)
            nlp_classifier.save(fname)
        start_idx = 0
    
    print('Success!')

In [ ]:
def test(data_x, nlp_embedding, nlp_classifier, label_encoder, batch_size, dataset_name, pred_dir='predictions', start_idx=0, fname_end=''):
    '''
    Test a model and return predictions.
    Parameters:
        data_x (Series): Lyrics of the test data.
        nlp_embedding (NLPEmbedding): Instance of the embedding method class.
        nlp_classifier (NLPClassifier): Instance of the classifier class.
        label_encoder (LabelEncoder): Label encoder after fitting.
        batch_size (int): Number of observations taken for single embedding and prediction.
        dataset_name (str): Name of the dataset.
        pred_dir (str): Name of directory for predictions.
        start_idx (int): In case of interrupted testing, from which observation restart the testing.
        fname_end (str): Suffix of the saved predictions file name.
    Returns:
        list: Predictions of genres.
    '''
    print('Testing...')
    fname = os.path.join(pred_dir, dataset_name, f'model_{nlp_embedding.name}_{nlp_classifier.name}{fname_end}.csv')
    predictions_all = []

    if start_idx == 0 and os.path.exists(fname):
        os.remove(fname)
    
    for i in range(start_idx, data_x.shape[0], batch_size):

        if i + batch_size > data_x.shape[0]:
            j = data_x.shape[0]
        else:
            j = i + batch_size
        
        print(f'Processing rows: {i} - {j - 1}')

        embeddings = nlp_embedding.embed_lyrics(data_x[i:j])
        predictions_enc = nlp_classifier.predict(embeddings)
        predictions = label_encoder.inverse_transform(predictions_enc)
        
        predictions_all.extend(predictions)

        pd.DataFrame(predictions.reshape(-1, 1)).to_csv(fname, mode='a', index=False, header=False)
    
    print('Success!')    
    
    return predictions_all

In [ ]:
def get_results(y_true, y_pred):
    '''
    Print accuracy, balanced accuracy and F1 score.
    Parameters:
        y_true (Series): True genres of the test data.
        y_pred (Series): Predicted genres of the test data.
    '''
    print('RESULTS:')
    print(f'accuracy = {metrics.accuracy_score(y_true=y_true, y_pred=y_pred)}')
    print(f'balanced accuracy = {metrics.balanced_accuracy_score(y_true=y_true, y_pred=y_pred)}')
    print(f'f1 score = {metrics.f1_score(y_true=y_true, y_pred=y_pred, average="weighted")}')

In [ ]:
def train_and_save_results(emb, clf, x_train, y_train, x_test, y_test, dataset_name, le, batch_size=5000, epochs=1, fname_end=''):
    '''
    Train a model, test it and print results.
    Parameters:
        emb (NLPEmbedding): Instance of the embedding method class.
        clf (NLPClassifier): Instance of the classifier class.
        x_train (Series): Lyrics of the training data.
        y_train (Series): Genres of the training data.
        x_test (Series): Lyrics of the test data.
        y_test (Series): Genres of the test data.
        dataset_name (str): Name of the dataset.
        le (LabelEncoder): Label encoder after fitting.
        batch_size (int): Number of observations taken for single embedding, partial fitting and predicting.
        epochs (int): Number of epochs.
        fname_end (str): Suffix of the saved model file name and saved predictions file name.
    '''
    train(x_train, y_train, emb, clf, le, batch_size, dataset_name, epochs=epochs, fname_end=fname_end)
    y_pred = test(x_test, emb, clf, le, batch_size, dataset_name, fname_end=fname_end)
    get_results(y_test, y_pred)
    return

In [ ]:
def add_normalized_lyrics(data):
    '''
    Add column to data frame with normalized lyrics created from tokens.
    Parameters:
        data (DataFrame): Data frame containing column with tokenized lyrics.
    '''
    tokens = data.tokens.apply(literal_eval)
    data['normalized_lyrics'] = [' '.join(t) for t in tokens]

In [ ]:
# Creation of necessary directories

model_dir = os.path.join('models', dataset_name)
if not os.path.exists(model_dir):
    os.makedirs(model_dir)

pred_dir = os.path.join('predictions', dataset_name)
if not os.path.exists(pred_dir):
    os.makedirs(pred_dir)

In [ ]:
# Reading training and test data from CSV files

train_data = pd.read_csv(f'data/train/{dataset_name}.csv')
test_data = pd.read_csv(f'data/test/{dataset_name}.csv')

In [ ]:
# Adding normalized lyrics based on tokens

add_normalized_lyrics(train_data)
add_normalized_lyrics(test_data)

In [ ]:
# Label encoder fitting

genres = np.unique(train_data.genre)
label_encoder = preprocessing.LabelEncoder()
label_encoder.fit(genres)
label_encoder.classes_

## Smaller BERT - training and testing

In [ ]:
emb_sm_bert = SmallBert(max_words)

### Unnormalized lyrics

In [ ]:
# Naive Bayes classifier

clf_nb = NaiveBayes()
train_and_save_results(emb_sm_bert, clf_nb, 
                       train_data.lyrics, train_data.genre, test_data.lyrics, test_data.genre, 
                       dataset_name, label_encoder)

In [ ]:
# Linear SVM classifier

clf_svm = SVM()
train_and_save_results(emb_sm_bert, clf_svm, 
                       train_data.lyrics, train_data.genre, test_data.lyrics, test_data.genre, 
                       dataset_name, label_encoder)

In [ ]:
# XGBoost classifier

clf_xgb = XGBoost(len(label_encoder.classes_))
train_and_save_results(emb_sm_bert, clf_xgb, 
                       train_data.lyrics, train_data.genre, test_data.lyrics, test_data.genre, 
                       dataset_name, label_encoder)

In [ ]:
# CNN classifier

clf_cnn = CNN(max_words * emb_sm_bert.embedding_size, len(genres), 'adam')
train_and_save_results(emb_sm_bert, clf_cnn,
                       train_data.lyrics, train_data.genre, test_data.lyrics, test_data.genre, 
                       dataset_name, label_encoder, epochs=cnn_epochs)

### Normalized lyrics

In [ ]:
# Naive Bayes classifier

clf_nb_norm = NaiveBayes()
train_and_save_results(emb_sm_bert, clf_nb_norm, 
                       train_data.normalized_lyrics, train_data.genre, test_data.normalized_lyrics, test_data.genre, 
                       dataset_name, label_encoder, fname_end='_norm')

In [ ]:
# Linear SVM classifier

clf_svm_norm = SVM()
train_and_save_results(emb_sm_bert, clf_svm_norm, 
                       train_data.normalized_lyrics, train_data.genre, test_data.normalized_lyrics, test_data.genre, 
                       dataset_name, label_encoder, fname_end='_norm')

In [ ]:
# XGBoost classifier

clf_xgb_norm = XGBoost(len(label_encoder.classes_))
train_and_save_results(emb_sm_bert, clf_xgb_norm, 
                       train_data.normalized_lyrics, train_data.genre, test_data.normalized_lyrics, test_data.genre, 
                       dataset_name, label_encoder, fname_end='_norm')

In [ ]:
# CNN classifier

clf_cnn_norm = CNN(max_words * emb_sm_bert.embedding_size, len(genres), 'adam')
train_and_save_results(emb_sm_bert, clf_cnn_norm,
                       train_data.normalized_lyrics, train_data.genre, test_data.normalized_lyrics, test_data.genre, 
                       dataset_name, label_encoder, epochs=cnn_epochs, fname_end='_norm')

## GloVe - training and testing

In [ ]:
emb_glove = GloVe(max_words)

### Unnormalized lyrics

In [ ]:
# Naive Bayes classifier

clf_nb = NaiveBayes()
train_and_save_results(emb_glove, clf_nb, 
                       train_data.lyrics, train_data.genre, test_data.lyrics, test_data.genre, 
                       dataset_name, label_encoder)

In [ ]:
# Linear SVM classifier

clf_svm = SVM()
train_and_save_results(emb_glove, clf_svm, 
                       train_data.lyrics, train_data.genre, test_data.lyrics, test_data.genre, 
                       dataset_name, label_encoder)

In [ ]:
# XGBoost classifier

clf_xgb = XGBoost(len(label_encoder.classes_))
train_and_save_results(emb_glove, clf_xgb, 
                       train_data.lyrics, train_data.genre, test_data.lyrics, test_data.genre, 
                       dataset_name, label_encoder)

In [ ]:
# CNN classifier

clf_cnn = CNN(max_words * emb_glove.embedding_size, len(genres), 'adam')
train_and_save_results(emb_glove, clf_cnn,
                       train_data.lyrics, train_data.genre, test_data.lyrics, test_data.genre, 
                       dataset_name, label_encoder, epochs=cnn_epochs)

### Normalized lyrics

In [ ]:
# Naive Bayes classifier

clf_nb_norm = NaiveBayes()
train_and_save_results(emb_glove, clf_nb_norm, 
                       train_data.normalized_lyrics, train_data.genre, test_data.normalized_lyrics, test_data.genre, 
                       dataset_name, label_encoder, fname_end='_norm')

In [ ]:
# Linear SVM classifier

clf_svm_norm = SVM()
train_and_save_results(emb_glove, clf_svm_norm, 
                       train_data.normalized_lyrics, train_data.genre, test_data.normalized_lyrics, test_data.genre, 
                       dataset_name, label_encoder, fname_end='_norm')

In [ ]:
# XGBoost classifier

clf_xgb_norm = XGBoost(len(label_encoder.classes_))
train_and_save_results(emb_glove, clf_xgb_norm, 
                       train_data.normalized_lyrics, train_data.genre, test_data.normalized_lyrics, test_data.genre, 
                       dataset_name, label_encoder, fname_end='_norm')

In [ ]:
# CNN classifier

clf_cnn_norm = CNN(max_words * emb_glove.embedding_size, len(genres), 'adam')
train_and_save_results(emb_glove, clf_cnn_norm,
                       train_data.normalized_lyrics, train_data.genre, test_data.normalized_lyrics, test_data.genre, 
                       dataset_name, label_encoder, epochs=cnn_epochs, fname_end='_norm')